In [1]:
import numpy as np
import torch

soap_features = torch.randn(10, 100).float()
soap_features.requires_grad = True
uncs = torch.zeros((10, 1), dtype=torch.float32)
for i in range(10):
    mu, kernel_inv = np.random.randn(100), np.random.randn(100, 100)
    soap_i = soap_features[i].reshape(1, -1)
    soap_i = (
        soap_i / torch.norm(soap_i, dim=1, keepdim=True)
    ) - torch.from_numpy(mu).float().reshape(1, -1)
    unc_i = torch.mm(soap_i, torch.from_numpy(kernel_inv).to(torch.float32)).mm(
        soap_i.t()
    )
    uncs[i] = unc_i
unc = torch.mean(uncs)
unc.backward()
grad = soap_features.grad
print(grad.shape)

torch.Size([10, 100])


In [5]:
import scipy.stats as stats

# 参数
feature_dim = 128  # SOAP 特征维度
confidence_level = 0.99  # 置信水平 (1 - α)

# 卡方分布的逆累积分布函数 (inverse CDF)
unc_threshold = stats.chi2.ppf(confidence_level, df=feature_dim)
print(f"Uncertainty threshold at {confidence_level*100}% confidence: {unc_threshold}")


Uncertainty threshold at 99.0% confidence: 168.13320306666492
